In [2]:
#importing libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import requests
from bs4 import BeautifulSoup
from csv import writer
import numpy as np
import pandas as pd
import os

In [3]:
#display setup for visibility
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
#url adress to scrap
page_info = 'https://europeanleague.football/live-games/220604-istanbul-rams-cologne-centurions'

In [5]:
#connecting with chrome 
service = Service('./chromedriver.exe')
service.start()
driver = webdriver.Remote(service.service_url)
driver.get(page_info)
time.sleep(3)
#clicking through cookies
driver.find_element(By.XPATH,'.//*[@id="consentDialog"]/div[2]/div[2]/div/div[2]/div/div[1]/div').click()
time.sleep(3)
#closing add
driver.find_element(By.XPATH,'/html/body/div[23]/div[2]/div[2]').click()
time.sleep(3)
#moving the slider to show the drives info
driver.find_element(By.XPATH,'/html/body/div[7]/div[2]/div[6]/a/div[2]').click()

In [6]:
#expanding details for all drives
input_element = driver.find_elements(By.XPATH,'.//*[starts-with(@id, "driveindex")]/div[2]/div[7]')
for item in input_element:
    item.click()
    time.sleep(1)
page_source = driver.page_source

In [7]:
#connecting to BS4
soup = BeautifulSoup(page_source, 'lxml')

In [11]:
#scrapping the data
lists_all = soup.find('div', id='drives_by_quarter_inner')

for item in lists_all:
    dfs = []
    lists_drive = lists_all.find_all('div', id=lambda x : x.startswith('driveindex') if x else False)
    for drive in lists_drive:
        elements = drive.find_all(class_="drives_plays_play")
        dane = []
        qtr = drive.find_previous_sibling("h3").text
        team = drive.find('img', class_='drives teamlogo')['name']
        drive_plays = drive.find('div', class_="drives_cell plays").find_next(text=True).strip()
        drive_yds = drive.find('div', class_="drives_cell yards").find_next(text=True).strip()
        drive_time = drive.find('div', class_="drives_cell totaltime").find_next(text=True).strip()
        columns = ['aa','play','bb','situation','dd','desc','cc']
        for element in elements:
            down = element.find('div', class_="drives_plays_play_next_step").text
            play = element.find('h4').text
            desc = element.find('div', class_="drives_plays_play_desc").text
            dane.append(element)
        
        df = pd.DataFrame(dane,columns=columns)
        df['qtr'] = qtr
        df['drive_num'] = drive['id']
        df['pos_team'] = team
        df['drive_plays'] = drive_plays
        df['drive_yds'] = drive_yds
        df['drive_time'] = drive_time
        dfs.append(df)
        
dfs = pd.concat(dfs)

C:\Users\iza\anaconda3\lib\site-packages\pandas\core\internals\construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [9]:
#droping not needed collumns
dfs.drop(['aa', 'bb','cc', 'dd'], axis=1,inplace=True)

In [10]:
#reseting and dropping index
dfs.reset_index(inplace=True)
dfs.drop(columns=['index'],inplace=True)

In [11]:
#splitting column 'desc' into 5 columns 
dfs[['down','team2','yds_to_go','team4','field_pos']] = pd.DataFrame(dfs.desc.tolist(), index = dfs.index)

In [12]:
#using expload on new columns which contains lists into objects 
dfs = dfs.explode('play')
dfs = dfs.explode('situation')
dfs = dfs.explode('down')
dfs = dfs.explode('yds_to_go')
dfs = dfs.explode('field_pos')

In [13]:
#droping some not needed empty columns
dfs.drop(['desc', 'team2','team4'], axis=1,inplace=True)

In [14]:
#getting data for 3 columns with constant data for the entire game (url) from the page_info (url)
dfs['home_team'] = page_info.split('-')[4].strip().capitalize()
dfs['away_team'] = page_info.split('-')[2].strip().capitalize()
dfs['game_date'] = page_info.split('-')[1].strip().split('/')[1].strip()

In [15]:
dfs

,play,situation,qtr,drive_num,pos_team,drive_plays,drive_yds,drive_time,down,yds_to_go,field_pos,home_team,away_team,game_date
0,Pass,J. Weinreich pass incomplete to Q. Pounds.,1st Quarter,driveindex_1,Cologne Centurions,3,1,1:21,1st,10,CC24,Cologne,Istanbul,220604
1,Rush,T. King rush for 1 yard to the CC25 (T. Bulusa...,1st Quarter,driveindex_1,Cologne Centurions,3,1,1:21,2nd,10,CC24,Cologne,Istanbul,220604
2,Pass,J. Weinreich pass incomplete to L. Geyer.,1st Quarter,driveindex_1,Cologne Centurions,3,1,1:21,3rd,9,CC25,Cologne,Istanbul,220604
3,Punt,"J. Appelt punt BLOCKED, recovered by CC TEAM a...",1st Quarter,driveindex_1,Cologne Centurions,3,1,1:21,4th,9,CC25,Cologne,Istanbul,220604
4,Kickoff,"J. Appelt kickoff 68 yards to the IR2, J. Mill...",1st Quarter,driveindex_1,Cologne Centurions,3,1,1:21,1st,0,CC30,Cologne,Istanbul,220604
5,Drive start,ISTANBUL RAMS drive start at 13:31.,1st Quarter,driveindex_2,Istanbul Rams,3,2,1:03,1st,10,IR22,Cologne,Istanbul,220604
6,Pass,J. Stegman pass complete to J. Conwell for 2 y...,1st Quarter,driveindex_2,Istanbul Rams,3,2,1:03,1st,10,IR22,Cologne,Istanbul,220604
7,Pass,J. Stegman pass incomplete (A. Naus).,1st Quarter,driveindex_2,Istanbul Rams,3,2,1:03,2nd,8,IR24,Cologne,Istanbul,220604
8,Pass,J. Stegman pass incomplete to K. Wedderburn.,1st Quarter,driveindex_2,Istanbul Rams,3,2,1:03,3rd,8,IR24,Cologne,Istanbul,220604
9,Punt,"Y. Guven punt 53 yards to the CC23, downed.",1st Quarter,driveindex_2,Istanbul Rams,3,2,1:03,4th,8,IR24,Cologne,Istanbul,220604


In [ ]:
#saving data to csv file
if not os.path.isfile('elf_plays_2022_scrap.csv'):
    dfs.to_csv('elf_plays_2022_scrap.csv', header='column_names')
else: # else it exists so append without writing the header
    dfs.to_csv('elf_plays_2022_scrap.csv', mode='a', header=False)